In [1]:
import pathlib
import random
import json
import tempfile

import sys
sys.path.append('../src')
from mediatools import ffmpeg
import mediatools

In [ ]:
compilation = ffmpeg.create_montage(
    video_files=list(pathlib.Path('/mnt/HDDStorage/').rglob('*.mp4')), 
    output_filename='compilation.mp4', 
    clip_ratio=10, 
    clip_duration=0.5, 
    verbose=False,
    max_clips_per_video=50,
)

In [4]:
fn = '/mnt/MoStorage/gopro/GoPro 2024-01-01/GX012082.MP4'
gopro_vf = mediatools.VideoFile(fn)
music_fn = list(pathlib.Path('music_vids/').rglob('*.mp4'))
music_vfs = [mediatools.VideoFile(f) for f in music_fn]

In [ ]:
compilation = ffmpeg.create_montage(
    video_files=music_fn, 
    output_filename='compilation_music.mp4', 
    clip_ratio=10, 
    clip_duration=2.0, 
    verbose=False,
)

In [4]:
gopro_src = pathlib.Path('/mnt/MoStorage/gopro/')


def get_gopro_paths(root: pathlib.Path|str, start: int, end: int) -> list[pathlib.Path]:
    root = pathlib.Path(root)
    all_paths = list(root.rglob('GX*.MP4')) + list(root.rglob('GX*.mp4'))
    all_paths.sort()
    paths = [(p, int(p.stem.split('_')[0][2:])) for p in all_paths]
    return [p for p, num in paths if start <= num <= end]


13242
use_paths = get_gopro_paths(gopro_src, start=12659, end=13242)
print(len(use_paths))
compilation = ffmpeg.create_montage(
    video_files=use_paths, 
    output_filename='gopro_compilation2.mp4', 
    clip_ratio=60, 
    clip_duration=3.0, 
    verbose=False,
)

430


In [ ]:
mediatools.ffmpeg.compress_video_by_bitrate(
    gopro_vf.fpath,
    pathlib.Path('compressed_gopro.mp4'),
    target_av_bitrate=2_500_000,
)